In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [ ]:
#lenth of the text

In [1]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

249892


In [ ]:
# First 2000 character of the text 

In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:20000])

The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Dorothy and the Wizard in Oz

Author: L. Frank Baum

Release date: February 1, 1996 [eBook #420]
                Most recently updated: January 1, 2021

Language: English

Credits: Produced by Dennis Amundson.


*** START OF THE PROJECT GUTENBERG EBOOK DOROTHY AND THE WIZARD IN OZ ***




Produced by Dennis Amundson.









Dorothy and the Wizard in Oz


  A Faithful Record of Their Amazing Adventures
    in an Underground World; and How with the
     Aid of Their Friends Zeb H

In [ ]:
#Calculates the number of unique characters — this is your vocabulary size.

In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [ ]:
#Encodeded 'hello'
# string  to intiger

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print (encode('hello'))

[64, 61, 68, 68, 71]


In [5]:
#encode to decocde "[64, 61, 68, 68, 71] " to "hello"
#Intiger to string 

In [8]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello =encode('hello')
decoded_hello = decode(encoded_hello)

print(decoded_hello)


hello


In [9]:
#Neural networks can't understand raw text like "The Wizard of Oz" — they only understand numbers (tensors). 
# So we need to:Convert characters to integers (encode)
#Convert integers back to characters (decode) — useful for generating text later

In [ ]:
#Converts the entire book text (now a list of integers) into a PyTorch tensor.
#Just prints the first 100 encoded characters (as numbers),

In [10]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([90, 48, 64, 61,  1, 44, 74, 71, 66, 61, 59, 76,  1, 35, 77, 76, 61, 70,
        58, 61, 74, 63,  1, 61, 30, 71, 71, 67,  1, 71, 62,  1, 32, 71, 74, 71,
        76, 64, 81,  1, 57, 70, 60,  1, 76, 64, 61,  1, 51, 65, 82, 57, 74, 60,
         1, 65, 70,  1, 43, 82,  0,  1,  1,  1,  1,  0, 48, 64, 65, 75,  1, 61,
        58, 71, 71, 67,  1, 65, 75,  1, 62, 71, 74,  1, 76, 64, 61,  1, 77, 75,
        61,  1, 71, 62,  1, 57, 70, 81, 71, 70])


In [ ]:
#This is a demo to show how the model will be trained to predict the next character.


In [ ]:
#Example: If your text is:

"hello wo"
#Then:

x = "hello w"

y = "ello wo"


| Input (context) | Target (next char) |
| --------------- | ------------------ |
| `'h'`           | `'e'`              |
| `'he'`          | `'l'`              |
| `'hel'`         | `'l'`              |
| `'hell'`        | `'o'`              |
| `'hello'`       | `' '`              |
| `'hello '`      | `'w'`              |
| `'hello w'`     | `'o'`              |


In [ ]:
#Understanding Context-Target Pair Creation for Language Modeling (Block Size = 8)
#✅ Purpose:
#To train a language model, we need to create (input_sequence → next_token) pairs. This code generates those examples.

In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)




when input is tensor([90]) target is tensor(48)
when input is tensor([90, 48]) target is tensor(64)
when input is tensor([90, 48, 64]) target is tensor(61)
when input is tensor([90, 48, 64, 61]) target is tensor(1)
when input is tensor([90, 48, 64, 61,  1]) target is tensor(44)
when input is tensor([90, 48, 64, 61,  1, 44]) target is tensor(74)
when input is tensor([90, 48, 64, 61,  1, 44, 74]) target is tensor(71)
when input is tensor([90, 48, 64, 61,  1, 44, 74, 71]) target is tensor(66)


In [ ]:
#. Device Selection for Computation

In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
# max_iters = 1000
# # eval_interval = 2500
# learning_rate = 3e-4
# eval_iters = 250

cpu


In [ ]:
#why use this code with purpose  benefit and real world analogy  

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


False


AssertionError: Torch not compiled with CUDA enabled

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[73,  1, 54, 72,  1, 58, 75, 58],
        [58, 69, 65, 62, 58, 57,  1, 28],
        [56, 58, 72, 72, 11,  0,  0,  3],
        [26, 74, 73,  1, 33,  1, 54, 66]], device='cuda:0')
targets:
tensor([[ 1, 54, 72,  1, 58, 75, 58, 71],
        [69, 65, 62, 58, 57,  1, 28, 68],
        [58, 72, 72, 11,  0,  0,  3, 40],
        [74, 73,  1, 33,  1, 54, 66,  1]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



af!KV1E.]S*IWOth
P(c8xFL;EBkltv()L]Mqs"(iUU*2v,k5Mqu!MkS,DR8;3 WNH2MkRJTTa5ji]; NZzgm[RED(3U;-9EN9LMK11M!Y1Dc74bn:MTA?')fcAE9o-E,p&TojQaEgAD0irY
, jOPDRjWH
(lFT)ga9J9JZR'X!zYufMw [[J﻿tncltGiRtR8VXx71﻿x'RDrpcp7V:OvQTj;3lPP-?6﻿7ke:vN8﻿,ZqcP47K?;.W0gn94K1﻿HR',1YEn:fsh
kZynPTTa0Zs tG1E,izU8(w;;n['.ZgFe*zY69h,)tc[)WgDu6_0TNEyampjL'bBV!hgCnb&ULI_PA0S]V]eq:JQjHz:
ou﻿EMHR"(4L
ZecMDx
k&vMcAEV2RHS;eq2Rj4[)qCpXa'oumKITTnezCY;gDGrS33?weVR!m1Qu94Kb&v,0"XQt5vJilPlj
OIO"J69
w ycv
b]2Jdjk.IusN*I)9﻿
Kf)Vw5ung5*2


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.857, val loss: 2.867
step: 250, train loss: 2.835, val loss: 2.860
step: 500, train loss: 2.826, val loss: 2.856
step: 750, train loss: 2.814, val loss: 2.825
3.0060837268829346


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [19]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


tesle?uKvmot spY
eoyErdB
xxfrem[o V8WLDl8Mkm'uz"Cdd
datys?1V!ourM5Cxz&THG8U2K.L_u.HLORuze ls lXtlel8M5J'B&RL)la.O16
Tb sL7&or
BW


 ithN:wk"th!OF :efaSB!!ardgn0kl pcly,[x0W5vSwooyw5.be:WP1k
,com﻿eard["f7&p9P0U0&Q6HWxR'[sizQ4C4,:"MK-R6YCvulexk0M5﻿6yEucoatallm.R"Sb"KKacop(6-1BS.L:GemsitQ4UL"magg.QHLG(0azar lPq'bzzgQBL::Bj7jneabM4VSQLar tud. PO5w]uldls8'pgv!BU3T7L*UHPokrriZ3-(. hY38G
ZdmpuVJns. dos"DomD.41,﻿HPPJJ)P24   gqplazsh.9c QyjM5O
d,:7Gx3gnaH,"w1﻿*"RLpPtWY_﻿,A
Srd a, ;diZKImThere?BSmJXwo Qnz
